# Pipeline test notebook

### Import requirements

In [144]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np

from ImageProcessing import ImageProcessing
from StereoMatcher import StereoMatcher

%matplotlib ipympl

### Load calibrations and other data

In [145]:
imageProcessing = ImageProcessing()
imageProcessing.loadMonoCalibrationResults()
imageProcessing.loadStereoCalibration()
imageProcessing.loadStereoRectify()

Reading from data/monoCalibration.json
Loaded mono calibration
Reading from data/stereoCalibration.json
Loaded stereo calibration
Reading from data/stereoRectify.json
Loaded stereo rectification data


### Create stereo matcher

In [146]:
stereoMatcher = StereoMatcher("SGBM", \
                vertical=True, createRightMatcher=False)
stereoMatcher.referenceDispToDepthMatrix(\
imageProcessing.dispToDepthMatrix)

imageProcessing.initUndistortRectifyMap()
#stereoMatcher.createDisparityWLSFilter()

Reading from data/parametersSGBM.json


### Loading images

In [147]:
path = "captures/testImages"
imageGlobL = sorted(glob.glob("".join([path, "/top_*", ".png"])))
imageGlobR = sorted(glob.glob("".join([path, "/bottom_*", ".png"])))
print ("Selections: 0-{}".format(len(imageGlobL)-1))

Selections: 0-7


### Select image pair and display

In [148]:
imageNumber = 5

imageL = cv2.imread(imageGlobL[imageNumber])
imageR = cv2.imread(imageGlobR[imageNumber])

cv2.imshow("left/right", numpy.hstack([imageL, imageR]))
cv2.waitKey(0)
cv2.destroyAllWindows()

### Convert to grayscale and undistort

In [149]:
imageProcessing.convertToGrayscale(imageL, imageR)
imageProcessing.undistortRectifyRemap(imageProcessing.grayImageL, \
                                        imageProcessing.grayImageR)

### View undistorted image

In [150]:
cv2.imshow("left/right undistort", numpy.hstack(\
    [imageProcessing.undistortImageL, imageProcessing.undistortImageL]))

cv2.imshow("horizontal epipolar", \
    imageProcessing.drawHorEpipolarLines(\
        imageProcessing.undistortImageL, imageProcessing.undistortImageR))

cv2.imshow("vertical epipolar", \
    imageProcessing.drawVertEpipolarLines(\
        imageProcessing.undistortImageL, imageProcessing.undistortImageR))
        
cv2.waitKey(0)
cv2.destroyAllWindows()

### Compute disparity map

In [151]:
stereoMatcher.computeDisparity(\
                grayImageL=imageProcessing.undistortImageL, \
                grayImageR=imageProcessing.undistortImageR)

stereoMatcher.clampDisparity()
stereoMatcher.applyClosingFilter()
#stereoMatcher.applyWLSFilterDisparity()

print ("minDisparity:", stereoMatcher.disparityMapL.min())
print ("maxDisparity:", stereoMatcher.disparityMapL.max())

minDisparity: 9.0
maxDisparity: 38.8125


### View disparity map

In [152]:
plt.figure()
plt.imshow(cv2.rotate(stereoMatcher.disparityMapL, \
    cv2.ROTATE_90_CLOCKWISE), cmap="gray")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute depth map

In [153]:
focalLength = imageProcessing.projectionMatrixL[0][0] # changes with rectify?
baseline = 32 # mm, measured irl

stereoMatcher.disparityMapL[stereoMatcher.disparityMapL==0] = 0.9
stereoMatcher.disparityMapL[stereoMatcher.disparityMapL==-1] = 0.9

depthMap = np.empty_like(stereoMatcher.disparityMapL)
depthMap = (focalLength*baseline)/stereoMatcher.disparityMapL[:]

print (stereoMatcher.disparityMapL.min())
print (stereoMatcher.disparityMapL.max())
print (depthMap.shape)
print (depthMap.min())
print (depthMap.max())

9.0
38.8125
(640, 360)
610.7979
2634.066


### View depth map

In [154]:
plt.figure()
plt.imshow(cv2.rotate(depthMap, cv2.ROTATE_90_CLOCKWISE), cmap="Greys")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute point cloud

In [155]:
points = cv2.reprojectImageTo3D(stereoMatcher.disparityMapL, \
                                stereoMatcher.dispToDepthMatrix)

# Reshaping to a list of 3D coordinates
pointCloud = points.reshape((points.shape[0]*points.shape[1],3))[0::16]\
                                                        .astype(np.int16)

print (points.shape)
print (pointCloud.shape)

print (points[0,0])
print (pointCloud[0])

(640, 360, 3)
(14400, 3)
[ -713.31146 -1168.7544   2763.0342 ]
[ -713 -1168  2763]


Filtering extreme points

In [156]:
pointCloud = pointCloud[pointCloud[:, 2]>pointCloud[:, 2].min()]
pointCloud = pointCloud[pointCloud[:, 2]<pointCloud[:, 2].max()]
pointCloud = pointCloud[pointCloud[:, 1]>pointCloud[:, 1].min()]
pointCloud = pointCloud[pointCloud[:, 1]<pointCloud[:, 1].max()]
pointCloud = pointCloud[pointCloud[:, 0]>pointCloud[:, 0].min()]
pointCloud = pointCloud[pointCloud[:, 0]<pointCloud[:, 0].max()]

print (pointCloud.shape)

(11671, 3)


Rotate point cloud to have y forward

In [157]:
pointRotMatrix = np.array([ [ 0,  0, -1],
                            [-1,  0,  0],
                            [ 0,  1,  0] ])

pointCloud = np.dot(pointCloud[:], pointRotMatrix)

print (pointCloud.shape)
print (pointCloud[0])

(11671, 3)
[309 731 141]


### View point cloud

In [158]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection = "3d")

ax.scatter(pointCloud[:,0], pointCloud[:,1], pointCloud[:,2], s=1)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")

# Camera axis begins at 0
ax.set_ylim(0,)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Voxelize point cloud

In [159]:
# From the matplotlib documentation:
# Prepare some coordinates
x, y, z = np.indices((8, 8, 8))

# Draw cuboids in the top left and bottom right corners, and a link between them
voxels = (x < 3) & (y < 3) & (z < 3)

ax = plt.figure().add_subplot(projection='3d')
ax.voxels(voxels)#, edgecolor='k')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [407]:
voxelSize = 10 # mm
voxelRange = np.array(
  [[pointCloud[:, 0].min(), pointCloud[:, 1].min(), pointCloud[:, 2].min()],\
  [pointCloud[:, 0].max(), pointCloud[:, 1].max(), pointCloud[:, 2].max()]])\
  .astype(np.int16)

print(voxelRange, "range")

voxelUnits = (((voxelRange[1]-voxelRange[0]) - \
            (voxelRange[1]-voxelRange[0]) % voxelSize)/10).astype(np.int16)
print(voxelUnits, "units")

voxelCenter = (voxelRange[0] + (voxelRange[1]-voxelRange[0])/2 - (voxelRange[0] + (voxelRange[1]-voxelRange[0])/2)%10).astype(np.int16)
print(voxelCenter, "center")

[[-500  668 -317]
 [ 318 2260  412]] range
[ 81 159  72] units
[-100 1460   40] center


In [413]:
voxelOffset = np.empty([3, *voxelUnits])

voxelUnitRange = (np.array(
  [[pointCloud[:, 0].min(), pointCloud[:, 1].min(), pointCloud[:, 2].min()],\
  [pointCloud[:, 0].max(), pointCloud[:, 1].max(), pointCloud[:, 2].max()]])\
  /voxelSize).astype(np.int16)

print(voxelRangeClamp)

voxelUnitIndices = np.indices(voxelUnits)

voxelOffset[2] = (voxelUnitIndices[2]+voxelUnitRange[0,2])*10
print(voxelOffset[2][0,0,:])

voxelOffset[1] = (voxelUnitIndices[1]+voxelUnitRange[0,1])*10
print(voxelOffset[1][0,:,0])

voxelOffset[0] = (voxelUnitIndices[0]+voxelUnitRange[0,0])*10
print(voxelOffset[0][:,0,0])

[[-50  66 -31]
 [ 31 226  41]]
[-310. -300. -290. -280. -270. -260. -250. -240. -230. -220. -210. -200.
 -190. -180. -170. -160. -150. -140. -130. -120. -110. -100.  -90.  -80.
  -70.  -60.  -50.  -40.  -30.  -20.  -10.    0.   10.   20.   30.   40.
   50.   60.   70.   80.   90.  100.  110.  120.  130.  140.  150.  160.
  170.  180.  190.  200.  210.  220.  230.  240.  250.  260.  270.  280.
  290.  300.  310.  320.  330.  340.  350.  360.  370.  380.  390.  400.]
[ 660.  670.  680.  690.  700.  710.  720.  730.  740.  750.  760.  770.
  780.  790.  800.  810.  820.  830.  840.  850.  860.  870.  880.  890.
  900.  910.  920.  930.  940.  950.  960.  970.  980.  990. 1000. 1010.
 1020. 1030. 1040. 1050. 1060. 1070. 1080. 1090. 1100. 1110. 1120. 1130.
 1140. 1150. 1160. 1170. 1180. 1190. 1200. 1210. 1220. 1230. 1240. 1250.
 1260. 1270. 1280. 1290. 1300. 1310. 1320. 1330. 1340. 1350. 1360. 1370.
 1380. 1390. 1400. 1410. 1420. 1430. 1440. 1450. 1460. 1470. 1480. 1490.
 1500. 1510. 1520. 

In [ ]:
#voxelOccupancy

In [442]:
x,y,z = 0,0,0
mask=np.empty(voxelUnits)
for xmin in range(voxelRange[0,0], voxelRange[1,0], voxelSize):
    xmax = xmin+voxelSize
    for ymin in range(voxelRange[0,1], voxelRange[1,1], voxelSize):
        ymax = ymin+voxelSize
        for zmin in range(voxelRange[0,2], voxelRange[1,2], voxelSize):
            zmax = zmin+voxelSize
            print(zmax)
            for point in pointCloud:
                if (point[0]<xmax and point[0]>xmin and point[1]<ymax and point[1]>ymin and point[2]<zmin):
                    mask[x,y,z] = True
                else:
                     mask[x,y,z] = False
            z+=1
        y+=1
    x+=1
print (mask)

-307
-297
-287
-277
-267
-257
-247
-237
-227
-217
-207
-197
-187
-177
-167
-157
-147
-137
-127
-117
-107
-97
-87
-77
-67
-57
-47
-37
-27
-17
-7
3
13
23
33
43
53
63
73
83
93
103
113
123
133
143
153
163
173
183
193
203
213
223
233
243
253
263
273
283
293
303
313
323
333
343
353
363
373
383
393
403
413


IndexError: index 72 is out of bounds for axis 2 with size 72